# Digit Recognizer using CNN

MNIST ("Modified National Institute of Standards and Technology") is the de facto “hello world” dataset of computer vision. Since its release in 1999, this classic dataset of handwritten images has served as the basis for benchmarking classification algorithms. As new machine learning techniques emerge, MNIST remains a reliable resource for researchers and learners alike.

## Importing necessary libraries

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pathlib

### Reading the dataset

In [3]:
shape = (28, 28, 1)
batch_size = 128

In [4]:
dtrain = pd.read_csv("../input/digit-recognizer/train.csv")
dtest = pd.read_csv("../input/digit-recognizer/test.csv")
dtrain.head()

In [5]:
# Checking the test data
dtest.head()

### Detecting Features and labels

In [6]:
labels = dtrain['label']
features = dtrain.drop(['label'], axis = 1)

In [7]:
features = np.asarray(features)
features.shape

In [8]:
features = features.reshape((42000, 28, 28))
features.shape

In [9]:
labels = tf.keras.utils.to_categorical(labels)

## Ploting samples of the dataset

In [10]:
plt.figure(figsize = (20, 4))
for i in range(16):
  plt.subplot(2, 8, i + 1)
  plt.imshow(features[i])
plt.show()

In [11]:
# Now let's create the model
def create_model():
    left_input = tf.keras.layers.Input(shape = shape)
    filter = 32
    x = left_input
    for i in range(3):
        x = tf.keras.layers.Conv2D(filter,3, activation = "relu", padding = 'same')(x)
        x = tf.keras.layers.Dropout(0.4)(x)
        x = tf.keras.layers.MaxPooling2D()(x)
    filter*=2
    right_input = tf.keras.layers.Input(shape = shape)
    y = right_input
    filter = 32
    for i  in range(3):
        y = tf.keras.layers.Conv2D(filter,3, activation = "relu", padding = 'same', dilation_rate = 2)(y)
        y = tf.keras.layers.Dropout(0.4)(y)
        y = tf.keras.layers.MaxPooling2D()(y)
        filter*=2
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    y = tf.keras.layers.GlobalAveragePooling2D()(y)
    model = tf.keras.layers.concatenate([x, y])
    model = tf.keras.layers.Dropout(0.4)(model)
    output = tf.keras.layers.Dense(10, activation = "softmax")(model)
    model = tf.keras.models.Model([left_input, right_input], output)
    return model

In [12]:
model = create_model()
model.summary()

### Ploting the Model

In [13]:
tf.keras.utils.plot_model(model, show_shapes = True)

In [14]:
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001), loss= "categorical_crossentropy", metrics = ['accuracy'])

In [15]:
history = model.fit([features, features], labels, batch_size=128, epochs=160, verbose=True, validation_split = 0.2,callbacks = [tf.keras.callbacks.EarlyStopping(patience= 20, monitor='val_loss', mode = 'min', restore_best_weights=True)])

## The Results

In [16]:
import matplotlib.pyplot as plt
plt.figure(figsize = (20, 5))
plt.plot(history.history['accuracy'], label = "accuracy")
plt.plot(history.history['val_accuracy'], label = "val_accuracy")
plt.legend()

In [17]:
plt.figure(figsize = (20, 5))
plt.plot(history.history['loss'], label = "loss")
plt.plot(history.history['val_loss'], label= "val_loss")
plt.legend()

## Model Evaluation

In [18]:
model.evaluate([features, features], labels, batch_size= 128)

In [20]:
features_test = np.asarray(dtest)
features_test.shape

In [21]:
features_test = features_test.reshape(features_test.shape[0], 28, 28)
y_pred = model.predict([features_test, features_test])
y_pred

In [22]:
y_pred = np.argmax(y_pred, axis = 1)
y_pred